# 弹幕游戏的操作序列分析

## 问题背景

游戏正在耗费人类越来越多的时间，所以对于人类在电脑游戏这一形式化的系统上的行为的研究，无论对于人类自身还是游戏的相关产业都是否有用。
而在对于游戏可能产生的数据，除了在现实中的响应外，最重要的应该是游戏内部的操作序列。很多游戏可以记录玩家的操作序列，
通常被称为“回放”（replay）。这些回放仅仅包含操作信息，可以重新被游戏读取模拟玩家的操作，以无损失的重现一场游戏的过程。
这与现在流行的通过共享对游戏录制的视频是完全不同的，视频不用具有游戏即可以播放，但仅能记录画面的信息，
而这些画面信息完全可以由大小小多了的操作序列通过运行游戏重造出来。早期网络的网速较慢，出于与现在视频流行类似的共享需求，
回放技术得到了发展。

电脑游戏回放中可能最多的，是即时战略游戏的回放，虽然存在一些相关的分析（此处应有对kaggle那个星际争霸回放研究的引用，以及引用的引用），但是
这些信息比较复杂，我们可以对比另外也许是“回放”很多的非电脑游戏，如国际象棋和围棋，可以以“棋谱”为形式记录它们，
有着它们行为空间固有的简单性。只需注意到这些游戏上的人工智能取得成就（此处引用alphago），就可以意识到这一点。对于类似的输入简单地游戏，
如那些游戏机上只通过手柄的有限的按键输入的游戏，人工智能也取得了类似的成就，如有名的DQN算法（引用Atari游戏论文），其必须基于输入的离散性。
我们这里也研究一种类似的游戏，即弹幕游戏（STG）。

弹幕游戏主要是玩家在操纵角色躲避敌方“弹幕”同时进行反击的游戏，其挑战性主要在于玩家的反应速度与根据弹幕运动规律进行预判（被称为“底力”），
在更深度的玩家那里，记忆特定关卡的弹幕特征也是重要的（称为“背板”）。有名的弹幕游戏系列包括雷电，虫姬等，我们这里研究玩家较多，
从而积累下的回放数也较多的东方project系列，其中的第10部，风神录作为具体研究的游戏。

游戏的回放文件批量下载自http://score.royalflare.net/ 该网站保存了过往玩家自发上传的回放，通常属于打出彩的那种。
我们主要选择其中Lunatic难度（最高难度）“梦B型”的202个回放文件，其跨度从2007到2017年。

## 原始数据类型与数据处理

回放文件是二进制的，其包含元信息-关卡信息，关卡信息又包含关卡的元信息与该关卡的操作序列，操作序列是整个回放文件的主要数据。

操作序列由一系列48bit的数据块构成，其表示一帧（frame，通常来说1秒=60frame）中的按键情况。我们大致的实验揭示了以下部分重要按键与数据的关系：

000000000000000000000000000100000000000000000000
                       ***** ***                pressing ctrl, right, left, down, up, shift,x,z
000000000001000000000000000101000000000000000000
       ***** ***                                press ctrl, right, left, down, up, shift,x,z
000000000000000000000000000001000000000000010000
                                       
                                       ***** ***release ctrl, right,left,down,up,shift,x,z
                                       
（这个显示的不对，有毒吧。。）

48bit的数据块可以看做48个01变量，这里展示了24个键位的含义，分别为表示当前帧按下，按中，松开ctrl,右，左，下，上，shift，x，z键。
其他键可能表示Esc和一些特殊按键的三种情况，不过未做实验。

为了解析这些二进制文件，我们编写了Python程序并传到GitHub上，https://github.com/yiyuezhuo/touhou-replay-decoder 
但其中一些具体的解析的“魔数”的提供是由  threp https://github.com/Fluorohydride/threp 进行的，不幸的是其中有一些bug而C语言不好调试，
所以我们实现了Python版并强化了API。

上述程序的一个功能是将二进制文件转成json文件，这也是我们将作为附件的内容。但代码里主要使用Python的pickle暂存的文件。
因为数据占用内存太多对后面的贝叶斯推断程序无法执行。解码程序本身可在上面的github页面查到，转换程序在explore_help.py中。

虽然做过全部数据的探索，但下面我们只会提到这样的数据，202个回放现在每一个对应6个关卡（stage）的秒序列，每秒对应一个四个变量的记录，
表示这秒60帧中按下shift,ctrl,x按键的帧数，与“绝对运动量”。

其中shift是慢速移动的键，按下这键表示当前的场景比较紧张，需要精细移动。ctrl是跳过对话的键，表明了此时处于特定的对话阶段。x是“灵击”，
表明此时是刷分时刻或危险时刻。绝对移动量是根据是否当前帧处于慢速移动还是正常移动，在上下左右按键情况加权下的指标。具体来说，该秒内
某帧按住shift时还按下“下”（此时也可以按其他键，类似计算），将在该指数中+2.0，若没有按下shift则加4.5，若并没有按下“下”则贡献0。
虽然计算了互相1s内相互抵消的相对移动量，但没有纳入分析。